Setting Up the Python Script

This script requires ChemApp for Python newer than 834.0.0 for all features to
work, as it is meant to showcase the new abilities. If you want to read up on
this example, there is a blog post on the GTT website that explains all features
used: <link -->

In [ ]:
from chemapp.friendly import StreamCalculation, Units, ThermochemicalSystem
from chemapp.core import AmountUnit, TemperatureUnit, PressureUnit, EnergyUnit, PhaseMatterState

# Set global units for consistency
Units.set(A=AmountUnit.tonne, T=TemperatureUnit.C, P=PressureUnit.bar, E=EnergyUnit.J)

# Load specialized thermochemical database for ferrous metallurgy
ThermochemicalSystem.load("steelmaking.cst")

Input Stream Definition

In [ ]:
# Define hot metal input (values in tonnes)
hot_metal = StreamCalculation.create_st_cfs(
    "HotMetal",
    ["Fe",      "C",        "Si",       "S",        "P",        "Mn"        ],
    [94.6225,   4.35,       0.505,      0.0225,     0.1,        0.4         ],
    T=1300.0
)

# Define oxygen for decarburization and oxidation reactions
oxygen = StreamCalculation.create_st_cfs("Oxygen", ["O2"], [5.0], T=25.0)

# Define steel scrap for temperature moderation and yield enhancement
scrap = StreamCalculation.create_st_cfs("Scrap", ["Fe"], [18.0], T=25.0)

# Define lime addition for slag formation and impurity sequestration
lime = StreamCalculation.create_st_cfs("Lime", ["CaO"], [2.5], T=25.0)

Primary Steelmaking: LD Converter Simulation

In [ ]:
# Execute adiabatic reaction simulation for converter process
converter_result = StreamCalculation.reaction_adiabatic(
    [hot_metal, oxygen, scrap, lime]
)

# Extract final temperature for process validation
print(f"Converter temperature: {converter_result.T:.1f} °C")

Alternative Converter Simulation with Added Heat

In [ ]:
# Diathermal reaction assuming 200 MJ heat input in the converter
converter_result = StreamCalculation.reaction_diathermal(
    [hot_metal, oxygen, scrap, lime],
    dH=200000.0,  # in J
)

# Extract final temperature for process validation
print(f"Converter temperature: {converter_result.T:.1f} °C")

Product Separation

In [ ]:
# Execute phase separation based on density thresholds
off_gas, slag, steel = converter_result.create_streams_by_density([1.0, 5.0])

print(f"Off-gas amount: {off_gas.A:.2f} tonnes")
print(f"Slag amount: {slag.A:.2f} tonnes")
print(f"Steel amount: {steel.A:.2f} tonnes")

# Separation strategy utilizing matter state
off_gas_alt = converter_result.create_stream_by_state(
    name="OffGas",
    state=PhaseMatterState.GAS
)

solids_only = converter_result.create_stream_by_state(
    name="Precipitates",
    state=PhaseMatterState.SOLID
)

# Separation strategy utilizing phase names
slag_liquid_and_solid = converter_result.create_stream(
    name="Slag", 
    phs_exclude=["gas_ideal", "LIQUID#1"]
)

steel_alt = converter_result.create_stream(
    name="Steel", 
    phs_include=["LIQUID#1"]
)


Secondary Metallurgy: Alloying

In [ ]:
# Define alloying materials
Al = StreamCalculation.create_st_cfs("Al", ["Al"], [0.15], T=25.0)
Fe_Si = StreamCalculation.create_st_cfs("Fe_Si", ["Si", "Fe"], [0.0238, 0.0102], T=25.0)
Fe_Mn = StreamCalculation.create_st_cfs("Fe_Mn", ["Mn", "Fe"], [0.0272, 0.0068], T=25.0)
Fe_Ti = StreamCalculation.create_st_cfs("Fe_Ti", ["Ti", "Fe"], [0.0221, 0.0119], T=25.0)

# Execute isothermal reaction simulation at steel temperature
alloying_result = StreamCalculation.reaction_isothermal(
    [steel, Al, Fe_Si, Fe_Mn, Fe_Ti], 
    T=steel.T
)

# Extract alloyed steel stream for subsequent processing
alloyed_steel = alloying_result.create_stream("AlloyedSteel")

Secondary Metallurgy: Vacuum Degassing

In [ ]:
# Define argon stream for inert gas purging
Ar = StreamCalculation.create_st_cfs("Argon", ["Ar"], [0.05], T=25.0)

# Execute low-pressure equilibrium calculation
degasser_result = StreamCalculation.reaction_isothermal(
    [alloyed_steel, Ar], 
    T=alloyed_steel.T, 
    P=0.01  # Vacuum condition (10 mbar)
)

# Separate gaseous phase from degassed steel
degassing_gas, degassed_steel = degasser_result.create_streams_by_density([1.0])